# Bike Network Data Cleanup
This notebook was developed within the ArcGIS Pro environment and will not run as a stand alone script outside of an ArcGIS Pro environment. To run this notebook and access project-related datasets, you will need to download the ArcGIS Pro project from box [here](https://mtcdrive.box.com/s/gy0u7jg4i1mwad9vqarzlkigg6qxbcvv). The data, and other tools are all contained within the project. You'll need download/pull this notebook from github then open within the project.

In [1]:
import arcpy
import os

#Define the workspace
arcpy.env.workspace = r"Z:\Documents\ArcGIS\Projects\Active Transportation\Active Transportation.gdb"

#Allow features and files to be overwritten 
arcpy.env.overwriteOutput = True

In [2]:
# Define a function to return unique values
def unique_values(table ,field):
    with arcpy.da.SearchCursor(table, [field]) as cursor:
        return sorted({row[0] for row in cursor if row[0] is not None})

In [3]:
# Create a function to re-map field values
def remap_field_values(fc,input_field,output_field,map_dictionary):
    update_fields = []
    update_fields.append(input_field)
    update_fields.append(output_field)
    with arcpy.da.UpdateCursor(in_table=fc,
                               field_names=update_fields) as cursor:
        for row in cursor:
            # store the Join value of the row being updated in a key variable
            key = row[0]
            # verify that the key is in the Dictionary
            if key in map_dictionary:
                row[1] = map_dictionary[key]
                cursor.updateRow(row)
    del cursor

In [4]:
#define a function for adding standard class fields & source
def add_required_fields(in_table):
    #Create list of lists with field name, data type, length (if applicable), and field alias
    list_of_lists = []
    source = ['source','TEXT','Source',100,None,None]
    existing_class = ['ex_class','SHORT','Existing Class',None,None,None]
    planned_class = ['pln_class','SHORT','Planned Class',None,None,None]
    list_of_lists.extend([source,existing_class,planned_class])
    #Add fields to feature class
    arcpy.management.AddFields(in_table,list_of_lists)

In [5]:
# Create a function to re-map multiple column values to one column
def remap_class_multi_columns(in_table,exist_field,plan_field,exist_map_dict,plan_map_dict):
    update_fields = []
    update_fields.extend([exist_field,plan_field,'ex_class','pln_class'])
    with arcpy.da.UpdateCursor(in_table=in_table,
                           field_names=update_fields) as cursor:
        for row in cursor:
            exist_key = row[0]
            pln_key = row[1]
            if (exist_key != None and exist_key != ' ') and (pln_key != None and pln_key != ' '):
                # store the Join value of the row being updated in a key variable for the existing field
                # verify that the key is in the Dictionary
                row[2] = exist_map_dict[exist_key]
                row[3] = plan_map_dict[pln_key]
                cursor.updateRow(row)
            elif (exist_key != None and exist_key != ' ') and (pln_key == None or pln_key == ' '):
                row[2] = exist_map_dict[exist_key]
                row[3] = None
                cursor.updateRow(row)
            elif (exist_key == None or exist_key == ' ') and (pln_key != None and pln_key != ' '):
                row[2] = None
                row[3] = plan_map_dict[pln_key]
                cursor.updateRow(row)
            else:
                row[2] = None
                row[3] = None
                cursor.updateRow(row)
    del cursor  

In [6]:
# Create a function to re-map class from one column to two columns depending on class value
def remap_class_single_to_multi_columns(in_table,class_field,status_field,class_map_dict,status_map_dict):
    update_fields = []
    update_fields.extend([class_field,status_field,'ex_class','pln_class'])
    with arcpy.da.UpdateCursor(in_table=in_table,
                               field_names=update_fields) as cursor:
        for row in cursor:
            class_key = row[0]
            status_key = row[1]
            if (class_key == None or class_key == ' ') and status_map_dict[status_key] == 'Existing':
                row[2] = 999
                cursor.updateRow(row)
            elif (class_key == None or class_key == ' ') and status_map_dict[status_key] == 'Planned':
                row[3] = 999
                cursor.updateRow(row)
            elif class_key != None and class_key != ' ' and status_map_dict[status_key] == 'Existing':
                row[2] = class_map_dict[class_key]
                cursor.updateRow(row)
            elif class_key != None and class_key != ' ' and status_map_dict[status_key] == 'Planned':
                row[3] = class_map_dict[class_key]
                cursor.updateRow(row)
            else:
                row[2] = None
                row[3] = None
                cursor.updateRow(row)
    del cursor  

In [7]:
# define a function to update the source field
def update_source(in_table,field_name,source):
    # update source column
    with arcpy.da.UpdateCursor(in_table=in_table,field_names=field_name) as source_cursor:
        for row in source_cursor:
            row[0] = source
            source_cursor.updateRow(row)
    del source_cursor

In [8]:
# Create function to select specific fields to map when using field mapping for a geoprocessing function
def issolate_fields(output_fields, field_mappings):
    for field in field_mappings.fields:
        if field.name not in output_fields:
            field_mappings.removeFieldMap(field_mappings.findFieldMapIndex(field.name))

In [9]:
# Create function to add only selected fields to field mapping object
def create_field_mapping_with_selected_fields(feature_classes,output_fields):
    fm = arcpy.FieldMappings()
    for fc in feature_classes:
        fm.addTable(fc)
        issolate_fields(output_fields=output_fields,field_mappings=fm)
    return fm

In [10]:
def merge_multiple_datasets(dataset_list,select_output_fields,output_fc):
    fm = create_field_mapping_with_selected_fields(feature_classes=dataset_list,
                                               output_fields=select_output_fields)
    print('Selected field list:')
    for field in fm.fields:
        print(field.name)
    print("""\n""")
            
    print("""Merging feature classes...\n""")
    merge_fc = arcpy.Merge_management(inputs=dataset_list,
                                           output=output_fc,
                                           field_mappings=fm,
                                           add_source='NO_SOURCE_INFO')
    print("""Merge complete""")
    return merge_fc

## Update Marin County Attributes

In [14]:
tam_bike_nw = 'tam_bike_network_epsg4326'
add_required_fields(in_table=tam_bike_nw)

In [15]:
# Get unique values for existingbi class field
exist_field = 'ExistingBi'
exist = unique_values(table=tam_bike_nw,field=exist_field)

In [16]:
# Get unique values for proposed class field
pln_field = 'ProposedBi'
planned = unique_values(table=tam_bike_nw,field=pln_field)

In [17]:
# Create dictonary with unique values as keys
dict.fromkeys(exist,'')

{' ': '', 'Class I': '', 'Class II': '', 'Class II / III Hybrid': '', 'Class II / IIIs Hybrid': '', 'Class III': '', 'Class IIIs': '', 'Class IIr': '', 'Class IIr / III Hybrid': '', 'FWY Rte': '', 'Other': ''}

In [18]:
# Create dictonary with unique values as keys
dict.fromkeys(planned,'')

{' ': '', 'Class I': '', 'Class II': '', 'Class III': '', 'Class IIr': '', 'Class IV (two-way)': ''}

In [19]:
# Create a dictionary to map existing facilities class to standard numeric class
exist_dict = {'Class I': 1, 
              'Class II': 2, 
              'Class II / III Hybrid': 2, 
              'Class II / IIIs Hybrid': 2, 
              'Class III': 3, 
              'Class IIIs': 3, 
              'Class IIr': 2, 
              'Class IIr / III Hybrid': 2, 
              'FWY Rte': 999, 
              'Other': 999}

In [20]:
planned_dict = {'Class I': 1, 
                 'Class II': 2, 
                 'Class III': 3, 
                 'Class IIr': 2, 
                 'Class IV (two-way)': 4}

In [21]:
# remap proposed class values to standard numeric class values
remap_class_multi_columns(in_table=tam_bike_nw,
                          exist_field=exist_field,
                          plan_field=pln_field,
                          exist_map_dict=exist_dict,
                          plan_map_dict=planned_dict)

In [22]:
# update source column
update_source(in_table=tam_bike_nw,
              field_name='source',
              source='Transportation Authority of Marin')

## Update Sonoma County Attributes

In [11]:
scta_bike_nw = 'scta_bike_network_epsg4326'
add_required_fields(in_table=scta_bike_nw)

In [12]:
# get unique class values
class_field = 'ORIG_CLASS'
classes = unique_values(table=scta_bike_nw,field=class_field)

In [13]:
# Get unique status values
status_field = 'GEN_PLAN'
status = unique_values(table=scta_bike_nw,field=status_field)

In [14]:
dict.fromkeys(classes,'')

{' ': '', 'I': '', 'II': '', 'IIB': '', 'III': '', 'III2': '', 'IIIB': '', 'IV': '', 'PED': '', 'PP': '', 'SR': '', 'STDY': '', 'TRL': ''}

In [18]:
dict.fromkeys(status,'')

{'': '', 'EXISTING': '', 'PROPOSED': ''}

In [19]:
class_dict = {'I': 1, 
              'II': 2, 
              'IIB': 2, 
              'III': 3, 
              'III2': 3, 
              'IIIB': 3, 
              'IV': 4, 
              'PED': 999, 
              'PP': 999, 
              'SR': 999, 
              'STDY': 999, 
              'TRL': 999}

In [22]:
status_dict = {'': None, 
              'EXISTING': 'Existing', 
              'PROPOSED': 'Planned'}

In [23]:
remap_class_single_to_multi_columns(in_table=scta_bike_nw,
                                    class_field=class_field,
                                    status_field=status_field,
                                    class_map_dict=class_dict,
                                    status_map_dict=status_dict)

In [24]:
# Update source column
update_source(in_table=scta_bike_nw,
              field_name='source',
              source='Sonoma County Transportation Authority')

## Update Napa County Attributes

In [41]:
nvta_bike_nw = 'nvta_bike_network_epsg4326'
add_required_fields(in_table=nvta_bike_nw)

In [42]:
# get unique class values
class_field = 'fac_type'
classes = unique_values(table=nvta_bike_nw,field=class_field)

In [43]:
# Get unique status values
status_field = 'orig_status'
status = unique_values(table=nvta_bike_nw,field=status_field)

In [44]:
dict.fromkeys(classes,'')

{'Class I Multi Use Path': '', 'Class II Bike Lane': '', 'Class III Bike Boulevard': '', 'Class III Bike Route': '', 'Class IV Separated Bike Lane': '', 'Corridor Study': ''}

In [45]:
dict.fromkeys(status,'')

{'Existing': '', 'Proposed': '', 'Proposed - pending study': ''}

In [46]:
class_dict = {'Class I Multi Use Path': 1, 
              'Class II Bike Lane': 2, 
              'Class III Bike Boulevard': 3, 
              'Class III Bike Route': 3, 
              'Class IV Separated Bike Lane': 4, 
              'Corridor Study': 999}

In [47]:
status_dict = {'Existing': 'Existing', 
              'Proposed': 'Planned', 
              'Proposed - pending study': 'Planned'}

In [48]:
remap_class_single_to_multi_columns(in_table=nvta_bike_nw,
                                    class_field=class_field,
                                    status_field=status_field,
                                    class_map_dict=class_dict,
                                    status_map_dict=status_dict)

In [49]:
# Update source column
update_source(in_table=nvta_bike_nw,
              field_name='source',
              source='Napa Valley Transportation Authority')

## Update Solano County Attributes

In [55]:
sta_bike_nw = 'sta_bike_network_epsg4326'
add_required_fields(in_table=sta_bike_nw)

In [56]:
# get unique class values
exist_class = 'existing'
classes = unique_values(table=sta_bike_nw,field=exist_class)

In [57]:
pln_class = 'rec'
planned = unique_values(table=sta_bike_nw,field=pln_class)

In [58]:
dict.fromkeys(classes,'')

{' ': '', 'Class I Multi-Use Path': '', 'Class II Bicycle Lane': '', 'Class II Buffered Bicycle Lane': '', 'Class III Bicycle Boulevard': '', 'Class III Bicycle Route': '', 'Class IV Separated Bikeway': ''}

In [59]:
dict.fromkeys(planned,'')

{' ': '', 'Class I Multi-Use Path': '', 'Class II Bicycle Lane': '', 'Class II Buffered Bicycle Lane': '', 'Class III Bicycle Boulevard': '', 'Class III Bicycle Route': '', 'Class IV Separated Bikeway': '', 'Feasibility Study': ''}

In [60]:
exist_dict = {
              'Class I Multi-Use Path': 1, 
              'Class II Bicycle Lane': 2, 
              'Class II Buffered Bicycle Lane': 2, 
              'Class III Bicycle Boulevard': 3, 
              'Class III Bicycle Route': 3, 
              'Class IV Separated Bikeway': 4}

In [61]:
prop_dict = { 
             'Class I Multi-Use Path': 1, 
             'Class II Bicycle Lane': 2, 
             'Class II Buffered Bicycle Lane': 2, 
             'Class III Bicycle Boulevard': 3, 
             'Class III Bicycle Route': 3, 
             'Class IV Separated Bikeway': 4, 
             'Feasibility Study': 999}

In [62]:
remap_class_multi_columns(in_table=sta_bike_nw,
                          exist_field=exist_class,
                          plan_field=pln_class,
                          exist_map_dict=exist_dict,
                          plan_map_dict=prop_dict)

In [63]:
# Update source column
update_source(in_table=sta_bike_nw,
              field_name='source',
              source='Solano Transportation Authority')

## Update Contra Costa Attributes

In [93]:
ccta_bike_nw = 'ccta_bike_network_epsg4326'
add_required_fields(in_table=ccta_bike_nw)

In [94]:
exist_class = 'class'
classes = unique_values(table=ccta_bike_nw,field=exist_class)

In [95]:
status_field = 'status'
status = unique_values(table=ccta_bike_nw,field=status_field)

In [96]:
dict.fromkeys(classes,)

{0: None, 1: None, 2: None, 3: None, 4: None, 5: None}

In [97]:
dict.fromkeys(status,)

{'Existing': None, 'Proposed': None}

In [98]:
# Typically we would not keep the 0 class but this should be kept for Contra Costa. 0 is any bike facility that is unpaved or dirt. 

exist_dict = {0: 0, 
              1: 1, 
              2: 2, 
              3: 3, 
              4: 4, 
              5: 999}

In [99]:
status_dict = {'Existing': 'Existing', 
               'Proposed': 'Planned'}

In [100]:
remap_class_single_to_multi_columns(in_table=ccta_bike_nw,
                                    class_field=exist_class,
                                    status_field=status_field,
                                    class_map_dict=exist_dict,
                                    status_map_dict=status_dict)

In [101]:
update_source(in_table=ccta_bike_nw,
              field_name='source',
              source='Contra Costa Transportation Authority')

## Update Alameda County Attributes

In [111]:
actc_bike_nw = 'actc_bike_network_epsg4326'
add_required_fields(in_table=actc_bike_nw)

In [112]:
# get unique class values
exist_field = 'existing_f'
classes = unique_values(table=actc_bike_nw,field=exist_field)

In [113]:
planned_field = 'planned_fa'
proposed = unique_values(table=actc_bike_nw,field=planned_field)

In [114]:
dict.fromkeys(classes,'')

{' ': '', 'Class I - Shared Use Path': '', 'Class II - Bike Lane': '', 'Class III - Shared Lane': '', 'Class IIIe - Bike Boulevard': '', 'Class IIb - Buffered Bike Lane': '', 'Class IV - Proteced Bike Lane': ''}

In [115]:
dict.fromkeys(proposed,'')

{' ': '', 'Class I - Shared Use Path': '', 'Class II - Bike Lane': '', 'Class III - Shared Lane': '', 'Class IIIe - Bike Boulevard': '', 'Class IIb - Buffered Bike Lane': '', 'Class IV - Proteced Bike Lane': '', 'Study Corridor': ''}

In [116]:
exist_dict = {'Class I - Shared Use Path': 1, 
              'Class II - Bike Lane': 2, 
              'Class III - Shared Lane': 3, 
              'Class IIIe - Bike Boulevard': 3, 
              'Class IIb - Buffered Bike Lane': 2, 
              'Class IV - Proteced Bike Lane': 4}

In [117]:
prop_dict = {'Class I - Shared Use Path': 1, 
             'Class II - Bike Lane': 2, 
             'Class III - Shared Lane': 3, 
             'Class IIIe - Bike Boulevard': 3, 
             'Class IIb - Buffered Bike Lane': 2, 
             'Class IV - Proteced Bike Lane': 4, 
             'Study Corridor': 999}

In [118]:
remap_class_multi_columns(in_table=actc_bike_nw,
                          exist_field=exist_field,
                          plan_field=planned_field,
                          exist_map_dict=exist_dict,
                          plan_map_dict=prop_dict)

In [119]:
# Update source column
update_source(in_table=actc_bike_nw,
              field_name='source',
              source='Alameda County Transportation Authority')

## Update Santa Clara County Attributes

In [120]:
vta_bike_nw = 'vta_bike_network_epsg4326'
add_required_fields(in_table=vta_bike_nw)

In [121]:
# get unique class values
class_field = 'BikeClassN'
classes = unique_values(table=vta_bike_nw,field=class_field)

In [122]:
# get unique status values
status_field = 'ExPrStatus'
status = unique_values(table=vta_bike_nw,field=status_field)

In [123]:
dict.fromkeys(classes,'')

{0: '', 1: '', 2: '', 3: '', 4: ''}

In [124]:
dict.fromkeys(status,'')

{'Existing': '', 'Proposed': ''}

In [125]:
exist_dict = {0: 999, 
              1: 1, 
              2: 2, 
              3: 3, 
              4: 4}

In [126]:
status_dict = {'Existing': 'Existing', 
               'Proposed': 'Planned'}

In [127]:
remap_class_single_to_multi_columns(in_table=vta_bike_nw,
                                    class_field=class_field,
                                    status_field=status_field,
                                    class_map_dict=exist_dict,
                                    status_map_dict=status_dict)

In [128]:
update_source(in_table=vta_bike_nw,
              field_name='source',
              source='Valley Transportation Authority')

## Update San Mateo County Attributes

In [129]:
ccag_bike_nw = 'ccag_bike_network_epsg4326'
add_required_fields(in_table=ccag_bike_nw)

In [130]:
exist_field = 'existing_facility'
exist = unique_values(table=ccag_bike_nw,
                      field=exist_field)

In [131]:
prop_field = 'final_recommended_facility'
proposed = unique_values(table=ccag_bike_nw,
                        field=prop_field)

In [132]:
dict.fromkeys(exist,'')

{'Class 1 Path': '', 'Class 2 Bicycle Lane': '', 'Class 2b Buffered Bicycle Lane': '', 'Class 3 Bicycle Route': '', 'Class 3b Bicycle Boulevard': '', 'Class 4 Separated Bicycle Lane': ''}

In [133]:
dict.fromkeys(proposed,'')

{'Class 1 Path': '', 'Class 2 Bicycle Lane': '', 'Class 2b Buffered Bicycle Lane': '', 'Class 3 Bicycle Route': '', 'Class 3b Bicycle Boulevard': '', 'Class 3c Bicycle Route with Wide Shoulders': '', 'Class 4 Separated Bicycle Lane': '', 'Keep Exising Facility': '', 'Undetermined Facility Type': ''}

In [134]:
exist_dict = {'Class 1 Path': 1, 
              'Class 2 Bicycle Lane': 2, 
              'Class 2b Buffered Bicycle Lane': 2, 
              'Class 3 Bicycle Route': 3, 
              'Class 3b Bicycle Boulevard': 3, 
              'Class 4 Separated Bicycle Lane': 4}

In [135]:
prop_dict = {'Class 1 Path': 1, 
             'Class 2 Bicycle Lane': 2, 
             'Class 2b Buffered Bicycle Lane': 2, 
             'Class 3 Bicycle Route': 3, 
             'Class 3b Bicycle Boulevard': 3, 
             'Class 3c Bicycle Route with Wide Shoulders': 3, 
             'Class 4 Separated Bicycle Lane': 4, 
             'Keep Exising Facility': None, 
             'Undetermined Facility Type': 999}

In [136]:
remap_class_multi_columns(in_table=ccag_bike_nw,
                          exist_field=exist_field,
                          plan_field=prop_field,
                          exist_map_dict=exist_dict,
                          plan_map_dict=prop_dict)

In [137]:
update_source(in_table=ccag_bike_nw,
              field_name='source',
              source='City/County Association of Governments San Mateo')

## Update San Francisco County Attributes

In [138]:
sfcta_bike_nw = 'sfcta_bike_network_epsg4326'
add_required_fields(in_table=sfcta_bike_nw)

In [139]:
#get unique class values
class_field = 'FACILITY_T'
classes = unique_values(table=sfcta_bike_nw,field=class_field)

In [140]:
dict.fromkeys(classes,'')

{'CLASS I': '', 'CLASS II': '', 'CLASS III': '', 'CLASS IV': '', 'CLASS iii': ''}

In [141]:
exist_dict = {'CLASS I': 1, 
              'CLASS II': 2, 
              'CLASS III': 3, 
              'CLASS IV': 4, 
              'CLASS iii': 3}

In [142]:
remap_field_values(fc=sfcta_bike_nw,
                   input_field=class_field,
                   output_field='ex_class',
                   map_dictionary=exist_dict)

In [143]:
#Update source column 

update_source(in_table=sfcta_bike_nw,field_name='source',
              source='San Francisco County Transportation Authority')

## Merge County Bike Networks

In [11]:
arcpy.ListFeatureClasses()

['regional_bikeway_network_2019_epsg4326', 'nvta_bike_network_epsg26910', 'sta_bike_network_epsg26910', 'sfcta_bike_network_epsg26910', 'actc_bike_network_epsg26910', 'vta_bike_network_epsg26910', 'ccta_bike_network_epsg26910', 'tam_bike_network_epsg29610', 'ccag_bike_network_epsg26910', 'scta_bike_network_epsg26910', 'regional_bike_facilities_2021_epsg26910', 'caltrans_d4_bike_network_epsg4326', 'regional_bike_facilities_2021_epsg4326', 'oakland_bike_network_epsg4326', 'batc_bike_network_epsg4326', 'vta_bike_network_epsg4326', 'tam_bike_network_epsg4326', 'sta_bike_network_epsg4326', 'sfcta_bike_network_epsg4326', 'scta_bike_network_epsg4326', 'nvta_bike_network_epsg4326', 'ccta_bike_network_epsg4326', 'ccag_bike_network_epsg4326', 'actc_bike_network_epsg4326']

In [12]:
dataset_list = ['nvta_bike_network_epsg4326', 
                'sta_bike_network_epsg4326', 
                'sfcta_bike_network_epsg4326', 
                'actc_bike_network_epsg4326', 
                'vta_bike_network_epsg4326', 
                'ccta_bike_network_epsg4326', 
                'tam_bike_network_epsg4326', 
                'ccag_bike_network_epsg4326', 
                'scta_bike_network_epsg4326']
output_fields = ['source','ex_class','pln_class']
regional_bike_facilities = 'regional_bike_facilities_2021_epsg4326'
merge_multiple_datasets(dataset_list=dataset_list,
                        select_output_fields=output_fields,
                        output_fc=regional_bike_facilities)

Selected field list:
source
ex_class
pln_class


Merging feature classes...

Merge complete


<Result 'Z:\\Documents\\ArcGIS\\Projects\\Active Transportation\\Active Transportation.gdb\\regional_bike_facilities_2021_epsg4326'>

## Change Regional Dataset Projection

In [167]:
regional_bike_facilities_4326 = 'regional_bike_facilities_2021_epsg4326'
transformation = 'NAD_1983_To_WGS_1984_5'
in_sr = arcpy.SpatialReference(26910)
out_sr = arcpy.SpatialReference(4326)
arcpy.Project_management(in_dataset=regional_bike_facilities,
                         out_dataset=regional_bike_facilities_4326,
                         in_coor_system=26910,
                         out_coor_system=4326,
                         transform_method=transformation)

<Result 'Z:\\Documents\\ArcGIS\\Projects\\Active Transportation\\Active Transportation.gdb\\regional_bike_facilities_2021_epsg4326'>

## Update Oakland Attributes

In [168]:
oak_bike_nw = 'oakland_bike_network_epsg4326'
add_required_fields(in_table=oak_bike_nw)

In [169]:
exist_field = 'existingcl'
exist = unique_values(table=oak_bike_nw,field=exist_field)

In [170]:
prop_field = 'proposedcl'
proposed = unique_values(table=oak_bike_nw,field=prop_field)

In [171]:
dict.fromkeys(exist,'')

{'0': '', '0.0': '', '1': '', '1X': '', '2': '', '2.0': '', '2.3': '', '2.3A': '', '2.3B': '', '2B': '', '2B.0': '', '2B.2': '', '2B.3A': '', '3': '', '3.0': '', '3A': '', '3A.0': '', '3A.3': '', '3B': '', '4': '', '4.2': '', '4.2B': '', '4.3A': ''}

In [172]:
dict.fromkeys(proposed,'')

{'0': '', '1': '', '2': '', '2.0': '', '2.3A': '', '2.3B': '', '2B': '', '2B.0': '', '2B.2': '', '2B.3A': '', '3': '', '3A': '', '3B': '', '4': ''}

In [173]:
exist_dict = {'0': None, 
              '0.0': None, 
              '1': 1, 
              '1X': 1, 
              '2': 2, 
              '2.0': 2, 
              '2.3': 2, 
              '2.3A': 2, 
              '2.3B': 2, 
              '2B': 2, 
              '2B.0': 2, 
              '2B.2': 2, 
              '2B.3A': 2, 
              '3': 3, 
              '3.0': 3, 
              '3A': 3, 
              '3A.0': 3, 
              '3A.3': 3, 
              '3B': 3, 
              '4': 4, 
              '4.2': 4, 
              '4.2B': 4, 
              '4.3A': 4}

In [174]:
prop_dict = {'0': None, 
             '1': 1, 
             '2': 2, 
             '2.0': 2, 
             '2.3A': 2, 
             '2.3B': 2, 
             '2B': 2, 
             '2B.0': 2, 
             '2B.2': 2, 
             '2B.3A': 2, 
             '3': 3, 
             '3A': 3, 
             '3B': 3, 
             '4': 4}

In [175]:
remap_class_multi_columns(in_table=oak_bike_nw,
                          exist_field=exist_field,
                          plan_field=prop_field,
                          exist_map_dict=exist_dict,
                          plan_map_dict=prop_dict)

In [176]:
update_source(in_table=oak_bike_nw,field_name='source',
              source='City of Oakland')

## Update Bay Area Trails Collaborative Attributes

In [25]:
batc_bike_nw = 'batc_bike_network_epsg4326'
add_required_fields(in_table=batc_bike_nw)

In [26]:
exist_field = 'TrailClass'
classes = unique_values(table=batc_bike_nw,field=exist_field)

In [27]:
# get unique status values
status_field = 'TrailStatus'
status = unique_values(table=batc_bike_nw,field=status_field)

In [28]:
dict.fromkeys(classes,'')

{'Class 1 bikeway': '', 'Class 2 bikeway': '', 'Class 3 bikeway': '', 'Class 4 bikeway': '', 'Multi-use trail': '', 'Pedestrian-only trail': '', 'TBD': '', 'Uncertain': ''}

In [29]:
dict.fromkeys(status,'')

{'Existing': '', 'Proposed': ''}

In [30]:
class_dict = {'Class 1 bikeway': 1, 
              'Class 2 bikeway': 2, 
              'Class 3 bikeway': 3, 
              'Class 4 bikeway': 4, 
              'Multi-use trail': 1, 
              'Pedestrian-only trail': 999, 
              'TBD': 999, 
              'Uncertain': 999}

In [31]:
status_dict = {'Existing': 'Existing', 
               'Proposed': 'Planned'}

In [34]:
remap_class_single_to_multi_columns(in_table=batc_bike_nw,
                                    class_field=exist_field,
                                    status_field=status_field,
                                    class_map_dict=class_dict,
                                    status_map_dict=status_dict)

In [35]:
update_source(in_table=batc_bike_nw,
              field_name='source',
              source='Bay Area Trails Collaborative')

## Update CalTrans D4 Attributes

In [183]:
caltrans_bike_nw = 'caltrans_d4_bike_network_epsg4326'
add_required_fields(in_table=caltrans_bike_nw)

In [184]:
class_field = 'class'
classes = unique_values(table=caltrans_bike_nw,field=class_field)

In [185]:
dict.fromkeys(classes,'')

{'I': '', 'II': '', 'IIB': '', 'IV': '', 'Shoulder': ''}

In [186]:
class_dict = {'I': 1, 
              'II': 2, 
              'IIB': 2, 
              'IV': 4, 
              'Shoulder': 999}

In [187]:
remap_field_values(fc=caltrans_bike_nw,
                   input_field=class_field,
                   output_field='pln_class',
                   map_dictionary=class_dict)

In [188]:
update_source(in_table=caltrans_bike_nw,
              field_name='source',
              source='California Department of Transportation D4')